# Tomography: getting started

In this notebook, you will learn how to use the QLM's tomography module, ``qat.tomo``.

## Hardware model definition

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from qat.core.circuit_builder.matrix_util import get_predef_generator, get_param_generator
import qat.mps
from qat.hardware import HardwareModel
from qat.hardware import GatesSpecification
from qat.noisy import NoisyQProc
from qat.quops import QuantumChannelPTM, QuantumChannelKraus, ParametricAmplitudeDamping
from qat.quops.converters import convert_kraus_to_ptm

with_ptm = True

if with_ptm:
    AD = QuantumChannelPTM(convert_kraus_to_ptm(ParametricAmplitudeDamping(T_1=10)(1).kraus_operators))
    ptm_dict = {key: AD*QuantumChannelPTM(convert_kraus_to_ptm([get_param_generator()[g_name](angle)]))
                for key, g_name, angle in [("X_PI2","RX", np.pi/2),
                                           ("Y_PI2", "RY", np.pi/2),
                                           ("X_PI", "RX", np.pi)]}

    target_gate_spec = GatesSpecification(gate_times={k: 0 for k in ptm_dict.keys()},
                                          quantum_channels=ptm_dict)

    qpu = NoisyQProc(hardware_model=HardwareModel(target_gate_spec),
                     sim_method="deterministic-vectorized")
else:
    AD = ParametricAmplitudeDamping(T_1=10)(1)
    kraus_dict = {key: QuantumChannelKraus([get_param_generator()[g_name](angle)])
                for key, g_name, angle in [("X_PI2","RX", np.pi/2),
                                           ("Y_PI2", "RY", np.pi/2),
                                           ("X_PI", "RX", np.pi)]}

    target_gate_spec = GatesSpecification(gate_times={k: 0 for k in kraus_dict.keys()},
                                          quantum_channels=kraus_dict)

    qpu = NoisyQProc(hardware_model=HardwareModel(target_gate_spec),
                     sim_method="deterministic-vectorized")

## Choosing a gate set, preparations and measurements

Tomography consists in using the QPU as a black box, with the only assumption that each quantum process, be it noise or a gate itself, acts as a linear map on the density matrix of the system. (One may also in addition assume that this map is trace-preserving and completely positive, but we will deal with these properties later on). To fully determine this matrix, one needs to prepare families of input states of rank at least the dimension of the underlying Hilbert space; similarly, one needs to determine the value of the projections of the final state on a family of independent measurements.

To prepare the input states and the measurements, the user just has to specify lists of gate sequences ${F_j}_j$ and ${F'_i}_i$ such that the corresponding input states $F_j |\rho_0\gg$ and measurements $\ll E| F'_i$ are *informationnally complete*, i.e are independent vectors in the Hilbert space. These two lists of gate sequences are sometimes called "preparation fiducials" and "measurement fiducials".

In the following cell, we prepare three different choices of fiducials.

In [ ]:
from qat.tomo.util import prepare_gatesets_and_fiducials
gatesets, prep_fiducials, meas_fiducials = prepare_gatesets_and_fiducials()


## Launching the tomography

Two distinct tomography methods are implemented in the QLM:
- **Quantum Process Tomography (QPT)** assumes that state preparations and final measurements are know *a priori*. Using this knowledge, one can use the collected tomography data to characterize the quantum channel one wants to determine. 
- **Gateset Tomography (GST)** handles state preparations and final measurements at the same level as noisy gates, and uses a self-consistent post-processing of the collected tomography data to characterize the full gate set, including the preparation and measurement operations.

In the QLM, tomography is performed by the function ``tomography``. The output is a ``DefaultGatesSpecification`` object containing the updated quantum channels (compared to the target gates).

In [ ]:
from qat.tomo import perform_tomography
from qat.quops.converters import convert_kraus_to_ptm
tomo_gates_spec = perform_tomography(qpu, 
                             method="lgst",  # which method to use (can also be "lgst")
                             prep_gates=prep_fiducials["gb1"],
                             meas_gates=meas_fiducials["gb1"],
                             target_gates_spec=target_gate_spec,  #expected gates (a priori knowledge, used by QPT)
                             #target_gates_spec=None,  #expected gates (a priori knowledge, used by QPT)
                             n_shots=0,  # number of runs for each circuit
                             qbits=[0],  # index of qubit(s) on which gate is applied
                             gate_list=["X_PI2","Y_PI2"],  # list of gate names to characterize
                             enforce_TP=False,
                             verbose=True)

# we can know print the corresponding quantum channels. Here, we use the Pauli Transfer Matrix (PTM) representation.
for gate_name in ["X_PI2"]:
    print("==== Gate ", gate_name)
    if with_ptm:
        G_tomo = tomo_gates_spec.quantum_channels[gate_name].ptm
        G_theo = target_gate_spec.quantum_channels[gate_name].ptm
    else:
        #G_tomo = convert_kraus_to_ptm(tomo_gates_spec._quantum_channels[gate_name].kraus_operators)
        G_tomo = tomo_gates_spec.quantum_channels[gate_name].ptm
        G_theo = convert_kraus_to_ptm(target_gate_spec.quantum_channels[gate_name].kraus_operators)
    print("G from tomography=\n",G_tomo)
    print("expected G=\n", G_theo)
    print("err = ", np.linalg.norm(G_tomo-G_theo))

We can know, using the noise model we just characterized, predict the outcome of a given circuit. Here, instead of using one of the QLM's simulators, we directly compute the final expectation value using the PTM representation of operators and superoperators:

In [ ]:
if with_ptm:
    E_theo = np.array([[0.5,0, 0, -0.5]])
    rho_theo = np.array([[0.5], [0], [0], [0.5]])
    E_tomo = tomo_gates_spec.meas# if tomo_gates_spec.meas is not None else E_theo
    rho_tomo = tomo_gates_spec.state_prep# if tomo_gates_spec.state_prep is not None else rho_theo

    print("<<E tomo| = ", E_tomo)
    print("| rho tomo>> = ", rho_tomo)

    for gate_name in ["X_PI2"]:
        print("==== Gate G = %s ========="% gate_name)
        G_tomo = tomo_gates_spec.quantum_channels[gate_name].ptm
        G_theo = target_gate_spec.quantum_channels[gate_name].ptm
        print("G tomo = ", G_tomo)
        print("G theor = ", G_theo)
        print("|0> --- X(pi/2)  ---> |0> - i|1>  --- (measure) ---> 0.5")
        print("tr(M G(rho_0))_tomo = ", 2*E_tomo.dot(G_tomo.dot(rho_tomo)))
        print("tr(M G(rho_0))_theo = ", 2*E_theo.dot(G_theo.dot(rho_theo)))
        print("|0> --- X(pi/2) X(pi/2)  ---> |1> --- (measure) ---> 1")
        print("tr(M G^2(rho_0))_tomo = ", 2*E_tomo.dot(G_tomo.dot(G_tomo.dot(rho_tomo))))
        print("tr(M G^2(rho_0))_theo = ", 2*E_theo.dot(G_theo.dot(G_theo.dot(rho_theo))))